In [1]:
import mindspore

In [2]:
import numpy as np
from mindspore import Tensor
from mindspore import dtype

In [3]:
x = Tensor(np.array([[1,2],[3,4]]), dtype.int32)
x

Tensor(shape=[2, 2], dtype=Int32, value=
[[1, 2],
 [3, 4]])

In [6]:
y = Tensor(1.0, dtype.int32)
z = Tensor(2, dtype.int32)
y

Tensor(shape=[], dtype=Int32, value= 1)

In [9]:
m = Tensor(True, dtype.bool_)
m

Tensor(shape=[], dtype=Bool, value= True)

In [10]:
n = Tensor((1,2,3), dtype.int16)
n

Tensor(shape=[3], dtype=Int16, value= [1, 2, 3])

In [11]:
p = Tensor([4.0, 5.0, 6.0], dtype.float64)
p

Tensor(shape=[3], dtype=Float64, value= [ 4.00000000e+00,  5.00000000e+00,  6.00000000e+00])

In [17]:
from mindspore import ops
oneslike = ops.OnesLike()
x = Tensor(np.array([[0,1],[2,1]]).astype(np.int32))
output  = oneslike(x)
output

Tensor(shape=[2, 2], dtype=Int32, value=
[[1, 1],
 [1, 1]])